### SETUP INICIAL DO PROJETO

In [ ]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account

# Carrega o .env: onde estão as credenciais do projeto/repositório
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")

# Detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"

# Parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
# Tables_to_process: lista de tabelas que serão processadas
# Output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.sales-salesorderdetail"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"

# Configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


In [ ]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

In [ ]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [ ]:
# Configuração para que o df exiba todas as colunas e todas as linhas completas, e também, exiba o formato numérico com 2 dígitos após a vírgula

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [ ]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    # Nome da tabela com substituição de '-' por '_'
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    # Ler os dados da tabela do BigQuery para um df
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    # Armazenar o df no dicionário
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")

# Print de validação
print("Todas as tabelas foram processadas com sucesso!")


In [ ]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

In [ ]:
# Atribuir o df a uma variável com nome mais simples
sales_salesorderdetail = df_processados['sales_salesorderdetail']

print(f"Colunas: {sales_salesorderdetail.shape[1]}\nLinhas: {sales_salesorderdetail.shape[0]}")

In [ ]:
# Identificar duplicadas com base em 'salesorderdetailid'
duplicadas = sales_salesorderdetail[
    sales_salesorderdetail.duplicated(subset=['salesorderdetailid'], keep=False)
]

# Verificar se existem duplicadas
if not duplicadas.empty:
    # Ordenar duplicadas por 'salesorderdetailid' e 'modifieddate'
    duplicadas_ordenadas = duplicadas.sort_values(by=['salesorderdetailid', 'modifieddate'])

    # Exibir duplicadas ordenadas
    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

In [ ]:
# Remover duplicadas mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
# Importante, pois se houver erro na ingestão (duplicação), mantém os dados integros.

sales_salesorderdetail = sales_salesorderdetail.drop_duplicates(subset=['salesorderdetailid'], keep='last')

print(f"Linhas após remover duplicadas (baseando-se na última 'modifieddate'): {len(sales_salesorderdetail)}")

#bkp dos dados brutos
raw_data_bkp_2_sem_duplicadas = sales_salesorderdetail.copy()


#doc: bkp = backup (cópia)

In [ ]:
# Ordenar e exibir o df por 'salesorderdetailid'
sales_salesorderdetail = sales_salesorderdetail.sort_values(by=['salesorderdetailid'])

print(sales_salesorderdetail)

In [ ]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in sales_salesorderdetail.columns:   
    missing_rows = sales_salesorderdetail[sales_salesorderdetail[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

In [ ]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = sales_salesorderdetail.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

#doc: currentflag possue somente 1 valor, o que indica que pode ser somente valores True ou False.

In [ ]:
#verificar informações do df
sales_salesorderdetail.info()

In [ ]:
# Variáveis quantitativas*: estatísticas descritivas para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['orderqty', 'unitprice','unitpricediscount']

# Estatísticas Descritivas das colunas numéricas*
print(sales_salesorderdetail[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)**
for col in numeric_columns:
    q1 = sales_salesorderdetail[col].quantile(0.25)
    q3 = sales_salesorderdetail[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers***
    outliers = sales_salesorderdetail[(sales_salesorderdetail[col] < lower_bound) | (sales_salesorderdetail[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


#doc*: variáveis quantitativas são um tipo de dado que pode ser representado por números e medidas objetivas, no nosso caso, vacationhours, sickleavehours, por exemplo
#doc*: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc**: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc***: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados

#doc****: as colunas analisadas apresentam outliers, porém os dados refletem operações reais da AdventureWorks, 
#         incluindo pedidos, preços, e descontos. Outliers em colunas como orderqty, unitprice, e unitpricediscount podem representar transações legítimas, 
#         como compras em grandes volumes, produtos premium, ou campanhas promocionais. Resultando em não manipular tais colunas.

In [ ]:
# Configurar o estilo do gráfico
sns.set(style='darkgrid', rc={"axes.facecolor": "black", "figure.facecolor": "black"})

# Lista das colunas para análise
columns_to_plot = ['orderqty', 'unitprice', 'unitpricediscount']

# Configurar a grade para 4 gráficos lado a lado
fig, axes = plt.subplots(1, len(columns_to_plot), figsize=(20, 12))  

# Criar os gráficos
for ax, col in zip(axes, columns_to_plot):
    # Calcular os quartis para a coluna
    quartiles = sales_salesorderdetail[col].quantile([0.25, 0.50, 0.75])

    # Gráfico de boxplot
    sns.boxplot(
        y=sales_salesorderdetail[col], ax=ax, showmeans=True,
        meanprops={"marker": "o", "markerfacecolor": "white", "markeredgecolor": "black", "markersize": 5},
        boxprops={"color": "green"}, whiskerprops={"color": "green"}, capprops={"color": "green"},
        medianprops={"color": "red"}, flierprops={"markerfacecolor": "blue", "markersize": 5}
    )

    # Título e rótulos
    ax.set_title(f'Distribuição de {col}', color='white', fontsize=14)
    ax.set_ylabel(col, color='white', fontsize=12)
    ax.set_xlabel("")
    ax.tick_params(axis='x', colors='white')
    ax.tick_params(axis='y', colors='white')

    # Adicionar os quartis como texto
    offset = 0.4
    for i, quartile in enumerate(quartiles):
        y_pos = quartile
        ax.text(x=0 + offset, y=y_pos, s=f"{quartile:.1f}", ha='left', va='center', fontsize=10, color='white')

# Ajustar layout
plt.tight_layout()
plt.show()


In [ ]:
# Histogramas*
sales_salesorderdetail[['orderqty', 'unitprice', 'unitpricediscount']].hist(
    bins=20, figsize=(12, 6), color='blue', edgecolor='black'
)
plt.suptitle("Distribuição de Variáveis Quantitativas", fontsize=16, color='white')
plt.grid(False)
plt.show()


#doc*: visualizar as distribuições de orderqty, unitprice e unitpricediscount

In [ ]:
# Agrupar por data e contar pedidos
time_analysis = sales_salesorderdetail.groupby(sales_salesorderdetail['modifieddate'].dt.date).size()

# Plotar evolução temporal*
plt.figure(figsize=(12, 6))
time_analysis.plot(color='green', linewidth=2)
plt.title("Evolução de Pedidos ao Longo do Tempo", fontsize=16, color='white')
plt.ylabel("Número de Pedidos", color='white')
plt.xlabel("Data", color='white')
plt.grid(False)
plt.show()


#doc*: entender como os dados evoluem ao longo do tempo

In [ ]:
# Contar pedidos com desconto e sem desconto*
discount_counts = sales_salesorderdetail['unitpricediscount'].value_counts()

# Plotar gráfico de barras
plt.figure(figsize=(8, 5))
discount_counts.plot(kind='bar', color='orange', edgecolor='black')
plt.title("Distribuição de Pedidos com e sem Desconto", fontsize=16, color='white')
plt.ylabel("Número de Pedidos", color='white')
plt.xlabel("Desconto", color='white')
plt.grid(False)
plt.show()


#doc*: verificar como os descontos (unitpricediscount) estão distribuídos

In [ ]:
# Produtos mais vendidos
top_products = sales_salesorderdetail['productid'].value_counts().head(10)

# Plotar gráfico de barras
plt.figure(figsize=(10, 5))
top_products.plot(kind='bar', color='cyan', edgecolor='black')
plt.title("Top 10 Produtos Mais Vendidos", fontsize=16, color='white')
plt.ylabel("Número de Vendas", color='white')
plt.xlabel("ID do Produto", color='white')
plt.grid(False)
plt.show()


#doc*: identificar os produtos mais vendidos

In [ ]:
# Análise cruzada entre 'specialofferid' e 'productid'*
offer_product = sales_salesorderdetail.groupby(['specialofferid', 'productid']).size().reset_index(name='counts')

# Exibir os 10 principais
print(offer_product.sort_values(by='counts', ascending=False).head(10))


#doc*: verificar a relação entre specialofferid e productid

In [ ]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['sales_salesorderdetail'] = sales_salesorderdetail

In [ ]:
# Padronizar colunas com valores textuais
sales_salesorderdetail['carriertrackingnumber'] = sales_salesorderdetail['carriertrackingnumber'].str.strip().str.upper()
sales_salesorderdetail['rowguid'] = sales_salesorderdetail['rowguid'].str.strip().str.upper()

print(sales_salesorderdetail.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

In [ ]:

# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
    # Nome da tabela no BigQuery
    output_table = f"{output_dataset}.{table_name}"

    # Configurar job de exportação
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    # Exportar DataFrame para o BigQuery
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")